# 🔬 Quantum-Classical Hybrid Computing Tutorial

**Author:** Meshal Alawein  
**Framework:** Optilibria 2.0 + ORCHEX 2.0

This tutorial demonstrates the quantum-classical hybrid optimization framework.

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '..')

# Import our quantum framework
from optilibria.optilibria import (
    QAOAOptimizer, VQEOptimizer, HybridOptimizer,
    QuantumSimulator, create_h2_hamiltonian
)

## 1. Quantum Circuit Basics

Let's start by creating a Bell state - the simplest entangled state.

In [ ]:
# Create a 2-qubit quantum simulator
sim = QuantumSimulator(2)

# Apply Hadamard to first qubit
sim.add_gate("H", [0])

# Apply CNOT (controlled-NOT)
sim.add_gate("CNOT", [0, 1])

# Get the state vector
state = sim.get_statevector()
print("Bell State |Φ+⟩ = (|00⟩ + |11⟩)/√2")
print(f"State vector: {state}")
print(f"\nProbabilities:")
for i, amp in enumerate(state):
    prob = np.abs(amp)**2
    if prob > 0.01:
        print(f"  |{i:02b}⟩: {prob:.3f}")

## 2. QAOA for Combinatorial Optimization

Solve the MaxCut problem using Quantum Approximate Optimization Algorithm.

In [ ]:
# Define a simple graph (square)
# Nodes: 0, 1, 2, 3
# Edges: 0-1, 1-2, 2-3, 3-0

def maxcut_cost(x):
    """Cost function for MaxCut - count edges between different partitions."""
    edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
    return -sum(x[i] != x[j] for i, j in edges)  # Negative for minimization

# Run QAOA with p=2 layers
qaoa = QAOAOptimizer(p=2)
result = qaoa.optimize(maxcut_cost, n_vars=4)

print("QAOA MaxCut Result:")
print(f"  Optimal partition: {result['x']}")
print(f"  Edges cut: {-result['fun']}")
print(f"  Iterations: {result['iterations']}")
print(f"\nInterpretation: Nodes {np.where(result['x']==0)[0]} vs Nodes {np.where(result['x']==1)[0]}")

## 3. VQE for Quantum Chemistry

Find the ground state energy of the H₂ molecule.

In [ ]:
# Create H2 Hamiltonian at equilibrium bond length
H, n_qubits = create_h2_hamiltonian(bond_length=0.74)

print(f"H₂ Hamiltonian ({n_qubits} qubits):")
print(H)

# Run VQE
vqe = VQEOptimizer(depth=2)
result = vqe.optimize(H, n_qubits)

# Compare with exact diagonalization
exact_energy = np.min(np.linalg.eigvalsh(H))

print(f"\nVQE Results:")
print(f"  VQE Energy: {result['energy']:.6f} Hartree")
print(f"  Exact Energy: {exact_energy:.6f} Hartree")
print(f"  Error: {abs(result['energy'] - exact_energy):.2e} Hartree")
print(f"  Iterations: {result['iterations']}")

## 4. Potential Energy Surface

Scan the H₂ bond length to find the equilibrium geometry.

In [ ]:
bond_lengths = np.linspace(0.4, 2.0, 20)
vqe_energies = []
exact_energies = []

vqe = VQEOptimizer(depth=2)

for r in bond_lengths:
    H, n_qubits = create_h2_hamiltonian(bond_length=r)
    
    # VQE
    result = vqe.optimize(H, n_qubits)
    vqe_energies.append(result['energy'])
    
    # Exact
    exact_energies.append(np.min(np.linalg.eigvalsh(H)))

# Find equilibrium
min_idx = np.argmin(vqe_energies)
print(f"Equilibrium bond length: {bond_lengths[min_idx]:.2f} Å")
print(f"Ground state energy: {vqe_energies[min_idx]:.4f} Hartree")

## 5. Hybrid Quantum-Classical Optimization

The HybridOptimizer automatically chooses between quantum and classical methods.

In [ ]:
# Rosenbrock function - a classic optimization benchmark
def rosenbrock(x):
    return sum(100*(x[i+1]-x[i]**2)**2 + (1-x[i])**2 for i in range(len(x)-1))

# Initialize hybrid optimizer
optimizer = HybridOptimizer()

# Starting point
x0 = np.array([-1.0, -1.0, -1.0])

# Optimize
result = optimizer.minimize(rosenbrock, x0)

print("Hybrid Optimizer Result:")
print(f"  Solution: {result.x}")
print(f"  Function value: {result.fun:.6f}")
print(f"  Iterations: {result.iterations}")
print(f"  Quantum advantage used: {result.quantum_advantage}")
print(f"\n  Expected minimum: [1, 1, 1] with f(x) = 0")

## 6. Grover's Search Algorithm

Quantum search with quadratic speedup.

In [ ]:
from optilibria.optilibria.quantum.grover import GroverSearch, create_search_oracle

# Search in a database of 16 items (4 qubits)
n_qubits = 4
N = 2**n_qubits

# Mark items 5 and 10 as targets
targets = [5, 10]
oracle = create_search_oracle(targets, n_qubits)

# Run Grover's search
grover = GroverSearch(n_qubits)
result = grover.search(oracle, num_solutions=2)

print(f"Grover's Search (N={N} items):")
print(f"  Targets: {targets}")
print(f"  Iterations: {result['iterations']} (classical would need ~{N//2})")
print(f"  Quantum speedup: {result['quantum_speedup']:.1f}x")
print(f"\nFound solutions:")
for sol in result['solutions']:
    print(f"  Index {sol['index']} (|{sol['bitstring']}⟩) - probability {sol['probability']:.3f}")

## 7. Physics Validation

Ensure all quantum operations respect fundamental physics laws.

In [ ]:
from optilibria.optilibria.physics.validation import PhysicsValidator, PhysicsLaw

validator = PhysicsValidator()

# Validate a quantum state
valid_state = np.array([1, 0, 0, 0], dtype=complex)  # |00⟩
invalid_state = np.array([1, 1, 0, 0], dtype=complex)  # Not normalized!

print("State Validation:")
result = validator.validate_quantum_state(valid_state)
print(f"  |00⟩: Valid = {result.valid}")

result = validator.validate_quantum_state(invalid_state)
print(f"  Unnormalized: Valid = {result.valid}")
print(f"    Violations: {result.violations}")

# Validate Hamiltonian
H, _ = create_h2_hamiltonian()
result = validator.validate_hamiltonian(H)
print(f"\nH₂ Hamiltonian: Hermitian = {result.valid}")

## Summary

This tutorial demonstrated:

1. **Quantum Circuit Simulation** - Bell states and basic gates
2. **QAOA** - Combinatorial optimization (MaxCut)
3. **VQE** - Quantum chemistry (H₂ molecule)
4. **Hybrid Optimization** - Automatic quantum/classical selection
5. **Grover's Search** - Quadratic speedup for unstructured search
6. **Physics Validation** - Ensuring physical correctness

### Next Steps

- Explore materials discovery with `qmatsim`
- Run autonomous research with `orchex`
- Scale to real quantum hardware with Qiskit/Cirq backends